### Loading data and libraries

In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 85%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from pprint import pprint
from scipy.stats import uniform
import statsmodels.api as sm
import math
import shap 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, cross_val_predict, GridSearchCV, RandomizedSearchCV, KFold, StratifiedKFold, validation_curve
from sklearn.pipeline import Pipeline as Pipeline, make_pipeline as make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier, Pool, cv
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.svm import SVC, LinearSVC
from sklearn.utils import resample
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE, RandomOverSampler, SMOTENC
from imblearn.under_sampling import NearMiss, RandomUnderSampler
from imblearn.pipeline import Pipeline as Imbpipeline
from imblearn.pipeline import make_pipeline as Imb_make_pipeline
from imblearn.combine import SMOTETomek, SMOTEENN
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectFromModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

df_data_set = pd.read_csv(r'C:\Users\moham\Dropbox\QSE\Thesis\Geopattern\My data\df_data_set_North_America_2a.csv')


In [ ]:
df_descriptive = df_data_set[['collaboration_binary', 'Geo_Dist', 'TENB', 'Cog_Dist', 'Prov_Border', 'Country_Border', 'NotContig']]
df_descriptive.describe()

In [ ]:
# Pearson correlation

df_descriptive = df_data_set[['collaboration_binary', 'Geo_Dist', 'TENB', 'Cog_Dist', 'Prov_Border', 'Country_Border', 'NotContig']]
df_descriptive.corr()

from scipy.stats import pearsonr
rho = df_descriptive.corr()
pval = df_descriptive.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
p = pval.applymap(lambda x: ''.join(['*' for t in [0.01,0.05,0.1] if x<=t]))
rho.round(2).astype(str) + p

### Regression analysis

In [ ]:
columns = [  'Log_Geo_Dist',
             'Log_TENB', 
             'Cog_Dist',
             'Prov_Border',
             'Country_Border',
             'NotContig',
             'Log_Geo_Dist X Log_TENB', 
             'Log_Geo_Dist_Sq X Log_TENB']


Ind_v = df_data_set[columns]
Dep_v = df_data_set.collaboration_binary

columns_reg1 = [ 'Log_Geo_Dist',
                 'Log_TENB',
                 'Cog_Dist',
                 'Prov_Border',
                 'Country_Border',
                 'NotContig']

columns_reg2 = [ 'Log_Geo_Dist',
                 'Log_TENB',
                 'Log_Geo_Dist X Log_TENB', 
                 'Cog_Dist',
                 'Prov_Border',
                 'Country_Border',
                 'NotContig']


X_reg1 = Ind_v[columns_reg1]
X_reg2 = Ind_v[columns_reg2]

X_reg_1 = sm.tools.tools.add_constant(X_reg1, prepend=True, has_constant='add')
X_reg_2 = sm.tools.tools.add_constant(X_reg2, prepend=True, has_constant='add')

models = [X_reg1, X_reg2]

for model in models:
    logit_model=sm.Logit(Dep_v, model)
    result=logit_model.fit(method_kwargs={"warn_convergence": False})
    print(result.summary2())

In [ ]:
# the geo distance that maximize the TENB's Elasticity

np.exp(1.0868/(2*0.0826))

In [ ]:
1.0868 * math.log1p(719.59) - 0.0826 *  math.log1p(719.59) *  math.log1p(719.59)

In [ ]:
dis = np.linspace (0,3000, num = 8691)

X_dis = []
for i in dis:
    x_dis = math.log1p(i)
    X_dis.append(x_dis)
    
Y_tenb = []
Y_tenb_ = []

for j in X_dis:
    y_tenb_ = 1.0868 * j - 0.0826 * j * j
    Y_tenb_.append(y_tenb_)

plt.figure(figsize=(20,10))
plt.plot (dis, Y_tenb_, color = 'red')
plt.xlabel('Physical distance (km)', size = 20)
plt.ylabel('Elasticity of TENB (network proximity)', size = 20)
plt.xticks(fontsize= 15)
plt.yticks(fontsize= 15)

### Machine Learning

In [2]:
features = ['Geo_Dist','TENB', 'Cog_Dist', 'Prov_Border', 'Country_Border', 'NotContig']

X = df_data_set[features]
y = df_data_set.collaboration_binary

In [3]:
unique, count = np.unique (y, return_counts = True)

y_value_count = {k : v for (k,v) in zip(unique,count)}

print ('Dataset', y_value_count)
print("\n")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1, stratify = y)

unique_train, count_train = np.unique (y_train, return_counts = True)

y_train_value_count = {k : v for (k,v) in zip(unique_train,count_train)}

print ('Dataset Train', y_train_value_count)
print("\n")

unique_test, count_test = np.unique (y_test, return_counts = True)

y_test_value_count = {k : v for (k,v) in zip(unique_test,count_test)}

print ('Dataset Test', y_test_value_count)
print("\n")


Dataset {0: 45605, 1: 333}


Dataset Train {0: 36484, 1: 266}


Dataset Test {0: 9121, 1: 67}




In [4]:
logreg = LogisticRegression(max_iter = 500)
gnb = GaussianNB()
knn = KNeighborsClassifier(n_jobs = -1)
svm = LinearSVC(loss = 'hinge')
rnd = RandomForestClassifier(random_state = 123, n_jobs = -1)
xgb = XGBClassifier(use_label_encoder=False, eval_metric = 'logloss', random_state = 123, n_jobs = -1)

rus = RandomUnderSampler()
ros = RandomOverSampler()
smt = SMOTENC(categorical_features = [3,4,5], random_state = 123, n_jobs = -1)

In [7]:
%%time
# Cross Validation

skf = StratifiedKFold(n_splits=5, shuffle = True, random_state = 123)

logreg_pipe_smt = Imbpipeline(steps = [['SMOTENC', smt], ['LogisticRegression', logreg]])
logreg_pipe_ros = Imbpipeline(steps = [['RandomOverSampler', ros], ['LogisticRegression', logreg]])
logreg_pipe_rus = Imbpipeline(steps = [['RandomUnderSampler', rus], ['LogisticRegression', logreg]])

gnb_pipe_smt = Imbpipeline(steps = [['SMOTENC', smt], ['GaussianNB', gnb]])
gnb_pipe_ros = Imbpipeline(steps = [['RandomOverSampler', ros], ['GaussianNB', gnb]])
gnb_pipe_rus = Imbpipeline(steps = [['RandomUnderSampler', rus], ['GaussianNB', gnb]])

knn_pipe_smt = Imbpipeline(steps = [['SMOTENC', smt], ['KNeighborsClassifier', knn]])
knn_pipe_ros = Imbpipeline(steps = [['RandomOverSampler', ros], ['KNeighborsClassifier', knn]])
knn_pipe_rus = Imbpipeline(steps = [['RandomUnderSampler', rus], ['KNeighborsClassifier', knn]])

svm_pipe_smt = Imbpipeline(steps = [['SMOTENC', smt], ['svm', svm]])
svm_pipe_ros = Imbpipeline(steps = [['RandomOverSampler', ros], ['svm', svm]])
svm_pipe_rus = Imbpipeline(steps = [['RandomUnderSampler', rus], ['svm', svm]])

rnd_pipe_smt = Imbpipeline(steps = [['SMOTENC', smt], ['Random Forest', rnd]])
rnd_pipe_ros = Imbpipeline(steps = [['RandomOverSampler', ros], ['Random Forest', rnd]])
rnd_pipe_rus = Imbpipeline(steps = [['RandomUnderSampler', rus], ['Random Forest', rnd]])

xgb_pipe_smt = Imbpipeline(steps = [['SMOTENC', smt], ['XGBoost', xgb]])
xgb_pipe_ros = Imbpipeline(steps = [['RandomOverSampler', ros], ['XGBoost', xgb]])
xgb_pipe_rus = Imbpipeline(steps = [['RandomUnderSampler', rus], ['XGBoost', xgb]])

clfs = [logreg, logreg_pipe_smt, logreg_pipe_ros, logreg_pipe_rus,
        gnb, gnb_pipe_smt, gnb_pipe_ros, gnb_pipe_rus,
        knn, knn_pipe_smt, knn_pipe_ros, knn_pipe_rus, 
        svm, svm_pipe_smt, svm_pipe_ros, svm_pipe_rus,
        rnd, rnd_pipe_smt, rnd_pipe_ros, rnd_pipe_rus,
        xgb, xgb_pipe_smt, xgb_pipe_ros, xgb_pipe_rus]

labels = ['logreg', 'logreg_pipe_smt', 'logreg_pipe_ros', 'logreg_pipe_rus',
          'gnb', 'gnb_pipe_smt', 'gnb_pipe_ros', 'gnb_pipe_rus',
          'knn', 'knn_pipe_smt', 'knn_pipe_ros', 'knn_pipe_rus', 
          'svm', 'svm_pipe_smt', 'svm_pipe_ros', 'svm_pipe_rus',
          'rnd', 'rnd_pipe_smt', 'rnd_pipe_ros', 'rnd_pipe_rus', 
          'xgb', 'xgb_pipe_smt', 'xgb_pipe_ros', 'xgb_pipe_rus']

       
clfs = zip(clfs,labels)

for clf, label in clfs:
    scores = cross_val_score(clf, X_train, y_train, scoring='f1', cv=skf, n_jobs=-1)
    print("F1: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))


F1: 0.68 (+/- 0.06) [logreg]
F1: 0.23 (+/- 0.02) [logreg_pipe_smt]
F1: 0.18 (+/- 0.01) [logreg_pipe_ros]
F1: 0.18 (+/- 0.01) [logreg_pipe_rus]
F1: 0.42 (+/- 0.04) [gnb]
F1: 0.16 (+/- 0.01) [gnb_pipe_smt]
F1: 0.31 (+/- 0.03) [gnb_pipe_ros]
F1: 0.28 (+/- 0.06) [gnb_pipe_rus]
F1: 0.68 (+/- 0.06) [knn]
F1: 0.27 (+/- 0.01) [knn_pipe_smt]
F1: 0.64 (+/- 0.03) [knn_pipe_ros]
F1: 0.06 (+/- 0.01) [knn_pipe_rus]
F1: 0.69 (+/- 0.05) [svm]
F1: 0.14 (+/- 0.16) [svm_pipe_smt]
F1: 0.12 (+/- 0.06) [svm_pipe_ros]
F1: 0.09 (+/- 0.07) [svm_pipe_rus]
F1: 0.78 (+/- 0.03) [rnd]
F1: 0.43 (+/- 0.02) [rnd_pipe_smt]
F1: 0.76 (+/- 0.03) [rnd_pipe_ros]
F1: 0.19 (+/- 0.02) [rnd_pipe_rus]
F1: 0.79 (+/- 0.02) [xgb]
F1: 0.42 (+/- 0.02) [xgb_pipe_smt]
F1: 0.76 (+/- 0.02) [xgb_pipe_ros]
F1: 0.16 (+/- 0.02) [xgb_pipe_rus]


In [8]:
%%time
# Hyperparameter tuning - RandomSearch

param_dist = { 'max_depth': [3, 5, 6, 10, 15, 20],
           'learning_rate': np.random.uniform(size=10),
           'subsample': np.random.uniform(size=10),
           'colsample_bytree': np.random.uniform(size=10),
           'colsample_bylevel': np.random.uniform(size=10),
           'n_estimators': [100, 500, 1000]}

skf = StratifiedKFold(n_splits=5, shuffle = True, random_state = 123)

xgb_srch_model = RandomizedSearchCV(estimator = xgb,
                           param_distributions = param_dist, 
                           cv = skf, n_jobs = -1, 
                           verbose = 1,
                           n_iter = 40,
                           scoring = 'f1',
                           random_state = 123)

xgb_srch_model.fit(X_train, y_train)
print ('Best Parameters', xgb_srch_model.best_params_)
print ('Best Score', xgb_srch_model.best_score_)

y_pred = xgb_srch_model.best_estimator_.predict(X_test)
print ('Presicion:' , precision_score(y_test, y_pred))
print ('Recall:' , recall_score(y_test, y_pred))
print ('F1_Score:', f1_score(y_test, y_pred))


Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best Parameters {'subsample': 0.26305272092626064, 'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.2667742254946316, 'colsample_bytree': 0.6123849685870691, 'colsample_bylevel': 0.4748701153315299}
Best Score 0.8038227414610967
Presicion: 0.8448275862068966
Recall: 0.7313432835820896
F1_Score: 0.784
Wall time: 5min 28s


In [ ]:
%%time
# Hyperparameter tuning - RandomSearch

param_dist = { 'max_depth': [3, 5, 6, 10, 15, 20],
           'learning_rate': np.random.uniform(size=10),
           'subsample': np.random.uniform(size=10),
           'colsample_bytree': np.random.uniform(size=10),
           'colsample_bylevel': np.random.uniform(size=10),
           'n_estimators': [100, 500, 1000]}

skf = StratifiedKFold(n_splits = 5)

xgb_srch_model = RandomizedSearchCV(estimator = xgb,
                           param_distributions = param_dist, 
                           cv = skf, n_jobs = -1, 
                           verbose = 1,
                           n_iter = 100,
                           scoring = 'f1',
                           random_state = 123)

xgb_srch_model.fit(X_train, y_train)
print ('Best Parameters', xgb_srch_model.best_params_)
print ('Best Score', xgb_srch_model.best_score_)

y_pred = xgb_srch_model.best_estimator_.predict(X_test)
print ('Presicion:' , precision_score(y_test, y_pred))
print ('Recall:' , recall_score(y_test, y_pred))
print ('F1_Score:', f1_score(y_test, y_pred))


In [ ]:
# Test scores
for clf, label in zip([logreg, gnb, knn, rnd, xgb, cat], ['LogisticRegression', 'GaussianNB', 'KNeighborsClassifier', 'Random Forest', 'XGB Classifier', 'CatBoostClassifier']):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
#     print("Presicion: %0.3f [%s]" % (precision_score(y_test, y_pred), label))
#     print("Recall: %0.3f [%s]" % (recall_score(y_test, y_pred), label))
    print("F1: %0.2f [%s]" % (f1_score(y_test, y_pred), label))
#     print("Accuracy: %0.2f [%s]" % (accuracy_score(y_test, y_pred), label))
#     print('\n')

In [ ]:
# baseline (RandomForest)

rus = RandomUnderSampler()
ros = RandomOverSampler()
smt = SMOTENC(categorical_features = [3,4,5], random_state = 123, n_jobs = -1)
scl = StandardScaler()

pipe_scl = Pipeline(steps = [['SCL', scl], ['RandomForestClassifier', xgb]])
pipe_smt = Imbpipeline(steps = [['SMOTENC', smt], ['RandomForestClassifier', xgb]])
pipe_ros = Imbpipeline(steps = [['RandomOverSampler', ros], ['RandomForestClassifier', xgb]])
pipe_rus = Imbpipeline(steps = [['RandomUnderSampler', rus], ['RandomForestClassifier', xgb]])

for clf, label in zip([xgb, pipe_scl, pipe_smt, pipe_ros, pipe_rus], ['No preprocess', 'SCL', 'SMOTENC', 'RandomOverSampler', 'RandomUnderSampler']):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
#     print("Presicion: %0.3f [%s]" % (precision_score(y_test, y_pred), label))
#     print("Recall: %0.3f [%s]" % (recall_score(y_test, y_pred), label))
    print("F1: %0.2f [%s]" % (f1_score(y_test, y_pred), label))
#     print("Accuracy: %0.3f [%s]" % (accuracy_score(y_test, y_pred), label))
#     print('\n')


In [ ]:
# Feature importance - mean decrease in impurity

importances = xgb.feature_importances_ * 100


df_importance = pd.DataFrame()

df_importance.insert(0,'Feature','')
df_importance.insert(1,'importance',0)

j = 0
for i in features:
    df_importance.loc[j,'Feature'] = i
    df_importance.loc[j,'importance'] = importances[j]
    j += 1

df_importance.sort_values(by=['importance'], ascending=False, inplace = True)

df_importance[:10].plot(x = 'Feature', y = 'importance', kind = 'bar', rot = 90, ylabel = 'mean decrease in impurity (%)')

In [ ]:
# Feature importance - SHAP

rnd.fit(X_train, y_train)

xgb_explainer = shap.TreeExplainer(rnd)
shap_values = xgb_explainer.shap_values(X_train, y_train)
shap_values2 = xgb_explainer(X_train)
shap_interaction_values = xgb_explainer.shap_interaction_values(X_train)


In [ ]:
shap.plots.bar(shap_values2)
shap.summary_plot(shap_values2, X_train, plot_type="bar")
shap.summary_plot(shap_values2, X_train, plot_size=2, cmap=plt.get_cmap("cool"))
# shap.plots.waterfall(shap_values2[0])


In [ ]:
# Dependence plot

shap.dependence_plot('Log_TENB', shap_values, X_train, interaction_index=None, show = False)

plt.gcf().set_size_inches(10, 10)

shap.dependence_plot(('Log_TENB', 'Log_Geo_Dist'), shap_interaction_values, X_train, show = False, cmap=plt.get_cmap("hsv"), dot_size = 70)

plt.gcf().set_size_inches(10, 10)
# ax = plt.axes()
# ax.set_facecolor("black")

# shap.dependence_plot('Cog_Dist', shap_values, X_train, interaction_index=None)
# shap.dependence_plot('Country_Border', shap_values, X_train, interaction_index=None)
# shap.dependence_plot('Prov_Border', shap_values, X_train, interaction_index=None)
# shap.dependence_plot('NotContig', shap_values, X_train, interaction_index=None)


In [ ]:
# Random Search Best parameters result (test data)

rnd_best = RandomForestClassifier(random_state = 123, n_jobs = -1,
                                       n_estimators = 170,
                                       min_samples_split = 5,
                                       min_samples_leaf = 1,
                                       max_features = 'sqrt',
                                       criterion = 'gini',
                                       max_depth = 55,
                                       bootstrap = True)

rnd_best.fit(X_train, y_train)
y_pred_best = rnd_best.predict(X_test)
print ('F1_Score:', f1_score(y_test, y_pred_best))
print ('Presicion:' , precision_score(y_test, y_pred_best))
print ('Recall:' , recall_score(y_test, y_pred_best))
print ('Accuracy:', accuracy_score(y_test, y_pred_best))
print ('confusion_matrix:')
print (confusion_matrix(y_test, y_pred_best))
print ('Classification Report:')
print (classification_report(y_test, y_pred_best))

# rfc_cv_score = cross_val_score(rnd_best, X_train, y_train, cv = skf, scoring = 'f1')

# print ('F1_Score_cv_score:', rfc_cv_score.mean())


In [ ]:
# Hyperparameter tuning - Randomforestclassifier - Grid search


n_estimators = [int(x) for x in np.linspace(start = 160, stop = 180, num = 3)]

max_features = ['auto','sqrt']

max_depth = [int(x) for x in np.linspace(53, 57, num = 5)]
max_depth.append(None)

min_samples_split = [4,5,6]

min_samples_leaf = [1]

bootstrap = [True, False]

criterion = ['gini', 'entropy']


param_grid = { 'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'criterion' : criterion}

print('Parameters used in the grid:\n')
pprint(param_grid)

skf = StratifiedKFold(n_splits = 5)

grid_model = GridSearchCV(estimator = rnd,
                         param_grid = param_grid, 
                         cv = skf, n_jobs = -1, 
                         verbose = 1,
                         scoring = 'f1')

grid_model.fit(X_train, y_train)

print ('Best Parameters', grid_model.best_params_)
print ('Best Score', grid_model.best_score_)


# Best Parameters {'bootstrap': True, 'criterion': 'gini', 'max_depth': 53, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 170}
# Best Score 0.8852592313348533

In [ ]:
# grid Search Best parameters result (test data)

grd_best = RandomForestClassifier(random_state = 123, n_jobs = -1,
                                       criterion = 'gini',
                                       n_estimators = 170,
                                       min_samples_split = 5,
                                       min_samples_leaf = 1,
                                       max_features = 'auto',
                                       max_depth = 53,
                                       bootstrap = True)

grd_best.fit(X_train, y_train)
y_pred_best_grd = grd_best.predict(X_test)
print ('F1_Score:', f1_score(y_test, y_pred_best_grd))
print ('Presicion:' , precision_score(y_test, y_pred_best_grd))
print ('Recall:' , recall_score(y_test, y_pred_best_grd))
print ('Accuracy:', accuracy_score(y_test, y_pred_best_grd))
print ('roc_auc_score:', roc_auc_score(y_test, y_pred_best_grd))
print ('confusion_matrix:')
print (confusion_matrix(y_test, y_pred_best_grd))
print ('Classification Report:')
print (classification_report(y_test, y_pred_best_grd))

# rfc_cv_score = cross_val_score(rnd_best, X_train, y_train, cv = skf, scoring = 'f1')

# print ('F1_Score_cv_score:', rfc_cv_score.mean())
